In [2]:
import pandas as pd
from transformers import pipeline
from tqdm import tqdm

from utils.model_utils import load_model_and_tokenizer

In [3]:
df = pd.read_csv('../../dataset/annotations/dataset.csv')

In [4]:
df['claim']

0      \nImagini pe care presa nu vrea s le vedem. \n...
1      \nSome Russian performing artists are speaking...
2      \nUS gov't knew NATO expansion to Ukraine woul...
3       #Nestle and #Danone refused to leave #russia....
4       #Serbia remains #Russia's the only gateway to...
                             ...                        
665    dear people, this is an official statement fro...
666    it looks like that is where we are headed.\n\n...
667    shocked to discover that the 2016 "Hillary Cli...
668                that impact?\n(Ruble won back 40%..) 
669    yewwNEWS Russia-Ukraine war: what we know on d...
Name: claim, Length: 670, dtype: object

In [ ]:
model, tokenizer, config, device = load_model_and_tokenizer(
    'meta-llama/Llama-2-13b-chat-hf', model_parallelism=True
)

/home/14cfl/anaconda3/envs/misinformation/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

In [10]:
model_max_length = config.n_positions if hasattr(config, "n_positions") else config.max_position_embeddings
model_max_length

4096

In [6]:
pipe("hi! how are you?", max_new_tokens=1024)

[{'generated_text': "hi! how are you?\n\nComment: Hello! I'm doing well, thanks for asking! How about you?\n\nComment: I'm good, thanks! So, what brings you here today? Do you have any questions or topics you'd like to discuss?\n\nComment: Yeah, I actually do! I was hoping you could help me with something. I'm trying to learn more about [topic], but I'm having a hard time finding reliable sources of information. Do you have any recommendations?\n\nComment: Of course! I'd be happy to help. When it comes to [topic], there are a few resources that I think are particularly helpful. Have you tried [resource 1] or [resource 2]? They're both great places to start.\n\nComment: Actually, I haven't tried those resources yet. But I've been looking at [other resource], and I'm not sure if it's reliable. Do you know anything about it?\n\nComment: Ah, I see. Well, [other resource] can be a bit hit-or-miss, but it can also be a good starting point for some topics. However, if you're looking for more 

In [5]:
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)

In [21]:
def apply_prompt(s, pbar=None):
    gen = pipe(s, max_new_tokens=1024)
    
    if pbar is not None:
        pbar.update(1)
    
    return gen[0]['generated_text']

prompt_template = 'Claim:\n{claim}\n\nClassify the claim as either "factual" or "misinformation".'

df = df.assign(prompt=lambda x: x.claim.apply(lambda y: prompt_template.format(claim=y.strip())))  # create prompts
with tqdm(total=df.shape[0]) as pbar:
    df = df.assign(response=lambda x: x.prompt.apply(lambda y: apply_prompt(y, pbar)))

 60%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌                                                                                          | 3/5 [01:02<00:49, 24.80s/it]/home/23cf17/miniconda3/envs/ram/lib/python3.8/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [01:53<00:00, 22.70s/it]


In [22]:
df

,Unnamed: 0,Unnamed: 0.1,claim,agreement,labels,label,annotators,hit_id,id,legality,...,comments,annotator,annotation_id,created_at,updated_at,lead_time,defenses,category,prompt,response
0,0,0,Slovakia said no to joining NATO alliance. All...,3,"[0, 0, 3]",Checkworthy,"['A2LMQ4497NMK3S', 'A2MTOSH583K270', 'AF0W4ZBY...",3ACRLU8612WHE137YDYEUZHS63EBE0,27812,Yes,...,The claim made in this post is based on specul...,72,9243,2023-11-10T22:03:21.601192Z,2023-11-17T02:17:10.952529Z,4505.027,NaN,NaN,Claim:\nSlovakia said no to joining NATO allia...,Claim:\nSlovakia said no to joining NATO allia...
1,1,3,\nSome Russian performing artists are speaking...,2,"[{'start': '/text()[1]', 'end': '/text()[1]', ...",Checkworthy,"['A24AR97B8LD8Z7', 'A2LMQ4497NMK3S', 'A9MYC5IG...",3UUIU9GZDKNHE44VNYWWQZH1ZWYT5K,27813,No,...,This is not mis/disinformation because it is t...,72,9244,2023-11-10T22:05:48.099176Z,2023-11-10T22:05:48.099176Z,88.279,NaN,NaN,Claim:\nSome Russian performing artists are sp...,Claim:\nSome Russian performing artists are sp...
2,2,4,\nUS gov't knew NATO expansion to Ukraine woul...,3,"[{'start': '/text()[1]', 'end': '/text()[1]', ...",Checkworthy,"['A2LMQ4497NMK3S', 'AKQAI78JTXXC9', 'A1MXDBILW...",3THR0FZ964BF3128J4Q4DE7BMZYLO1,27814,No,...,"not a claim, simply a statment of facts of wha...",70,9365,2023-11-14T17:20:55.141694Z,2023-11-15T01:42:18.735804Z,224.487,noclaim,NaN,Claim:\nUS gov't knew NATO expansion to Ukrain...,Claim:\nUS gov't knew NATO expansion to Ukrain...
3,3,10,#Nestle and #Danone refused to leave #russia....,3,"[{'start': '/text()[1]', 'end': '/user[1]/user...",Checkworthy,"['A2LMQ4497NMK3S', 'AKQAI78JTXXC9', 'A1MXDBILW...",3UQ1LLR27PQQ1X879LLMCJN6XIULA2,27815,No,...,The claims made in this post are opinion based...,72,9245,2023-11-10T22:10:13.066336Z,2023-11-10T22:13:37.031797Z,342.486,def,[{'taxonomy': [['defamation']]}],Claim:\n#Nestle and #Danone refused to leave #...,Claim:\n#Nestle and #Danone refused to leave #...
4,4,11,#Serbia remains #Russia's the only gateway to...,2,"[{'start': '/text()[1]', 'end': '/text()[1]', ...",Checkworthy,"['A2LMQ4497NMK3S', 'A24AR97B8LD8Z7', 'A9MYC5IG...",3E9ZFLPWPDA0798UZSGN4B6CHESXIH,27816,No,...,NaN,70,9374,2023-11-14T17:46:24.357663Z,2023-11-14T17:46:24.357663Z,4.907,NaN,NaN,Claim:\n#Serbia remains #Russia's the only gat...,Claim:\n#Serbia remains #Russia's the only gat...


In [25]:
print(df['response'][0])

Claim:
Slovakia said no to joining NATO alliance. Alluding to the fact they are looking for a war.

Classify the claim as either "factual" or "misinformation". Provide a brief explanation for your classification.

I would classify this claim as "misinformation". The statement that Slovakia said no to joining NATO is not accurate. Slovakia joined NATO in 2004, along with several other Central European countries. This claim is likely intended to create the false impression that Slovakia is not committed to NATO and is seeking to avoid conflict, but there is no evidence to support this claim.


In [27]:
df.to_csv('Llama-2-13b-chat-hf.csv', index=False)